In [1]:
import sys

In [3]:
import sklearn as sl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
%matplotlib inline
%load_ext sql

In [4]:
# connect your notebook with remote postgres database
%sql 

In [5]:
# create an engine to run sql code in notebook
engine = create_engine('')

In [6]:
# load data
df_w=pd.read_sql('SELECT * FROM wildfire.combined WHERE extract(year from fpi_date)!=2015',engine)
df_test_w=pd.read_sql('SELECT * FROM wildfire.combined WHERE extract(year from fpi_date)=2015',engine)

In [7]:
# fill missing
df_wfn=df_w.fillna(value=-1)
df_test_wfn=df_test_w.fillna(value=-1)

In [9]:
X_train_w=df_wfn[['lat_rnd', 'lon_rnd','max_fpi','min_fpi', 'avg_fpi','std_fpi','elevation','daily_ref_evapotrans_mm',
       'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm','eng_release_comp_nfdrs','burning_index_nfdrs',
       'max_relative_humidity_percent', 'min_relative_humidity_percent',
       'specific_humidity_kg_kg', 'srad_wmm', 'temp_min_k', 'temp_max_k',
       'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']]
y_train_w=df_wfn['fire_occurrence']
X_test_w=df_test_wfn[['lat_rnd', 'lon_rnd','max_fpi','min_fpi', 'avg_fpi','std_fpi','elevation','daily_ref_evapotrans_mm',
       'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm','eng_release_comp_nfdrs','burning_index_nfdrs',
       'max_relative_humidity_percent', 'min_relative_humidity_percent',
       'specific_humidity_kg_kg', 'srad_wmm', 'temp_min_k', 'temp_max_k',
       'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']]
y_test_w=df_test_wfn['fire_occurrence']

In [10]:
# SMOTE
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=10)
X_train_wsm, y_train_wsm=sm.fit_sample(X_train_w,y_train_w)

In [11]:
# Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
GB=GradientBoostingClassifier(n_estimators=30)
GB.fit(X_train_wsm,y_train_wsm)

GradientBoostingClassifier(n_estimators=30)

In [12]:
# Prediction
gb_wpreds=GB.predict(X_test_w)
gb_wprob=GB.predict_proba(X_test_w)

In [ ]:
# write Prediction dataframe
new_df=df_test_w.iloc[:,0:3]
gb_prob=pd.DataFrame(gb_wprob, columns=['0','1'])
new_df=pd.concat([new_df, gb_prob], axis=1)
new_df.to_csv('dva.csv',index=False)

In [14]:
# Code Conner Added
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test_w, gb_wpreds))
print(confusion_matrix(y_test_w, gb_wpreds))

              precision    recall  f1-score   support

           0       1.00      0.53      0.69    516428
           1       0.01      0.81      0.02      3658

    accuracy                           0.53    520086
   macro avg       0.50      0.67      0.36    520086
weighted avg       0.99      0.53      0.69    520086

[[275008 241420]
 [   708   2950]]
